In [ ]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# Importing the data

df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
df.columns = ['Sentence', 'Score']

Create a list of words that are in the negative and positive comments.

In [ ]:
negative_words = pd.DataFrame([])
positive_words = pd.DataFrame([])
lower_no_punc = pd.DataFrame([])

for index in df.index:
    #print(df.iloc[index]['Score'])
    if df.iloc[index]['Score'] == 0:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[^\w\d\s]+', '', s)
        negative_words = negative_words.append(out.lower().split())

    if df.iloc[index]['Score'] == 1:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[^\w\d\s]+', '', s)
        positive_words = positive_words.append(out.lower().split())
        #lower_no_punc = lower_no_punc.append([out.lower()])
    lower_no_punc = lower_no_punc.append([out.lower()])

lower_no_punc = lower_no_punc.reset_index(drop=True)
lower_no_punc.columns = ['Sentence']
negative_words = pd.DataFrame(negative_words[0].unique())
positive_words = pd.DataFrame(positive_words[0].unique())

Dropping out the words that are in both positive and negative lists.

In [52]:
# Compare word lists to each other and remove matching.
# print(negative_words, positive_words)

neg_match = negative_words[0].isin(positive_words[0])
pos_match = positive_words[0].isin(negative_words[0])

for index in neg_match.index:
    if neg_match.iloc[index] == True:
        negative_words.drop(index, inplace=True)

for index in pos_match.index:
    if pos_match.iloc[index] == True:
        positive_words.drop(index, inplace=True)
        
negative_words = negative_words.reset_index(drop=True)
positive_words = positive_words.reset_index(drop=True)

Outputting 

In [62]:
test = lower_no_punc[:5]
print(test['Sentence'][3])
for neg in negative_words[0]:
    print(neg)
    #result = test['Sentence'].str.contains(str(neg), case=False, regex=False)
    if lower_no_punc['Sentence'].str.contains(str(neg)).any():
        print('True')
    else:
        print('False')
    #print(result)
#     if any(result == True):
#         test.loc['Neg'] = True
#print(test)
    #         lower_no_punc['Neg'] = lower_no_punc['Sentence'].str.contains(str(neg), case=False, regex=False)
    
# for pos in positive_words[0]:
#     lower_no_punc['Pos'] = lower_no_punc['Sentence'].str.contains(str(neg), case=False, regex=True)

tied to charger for conversations lasting more than 45 minutesmajor problems
unless
RangeIndex(start=0, stop=5, step=1)
True
converter
RangeIndex(start=0, stop=5, step=1)
True
tied
RangeIndex(start=0, stop=5, step=1)
True
conversations
RangeIndex(start=0, stop=5, step=1)
True
lasting
RangeIndex(start=0, stop=5, step=1)
True
45
RangeIndex(start=0, stop=5, step=1)
True
minutesmajor
RangeIndex(start=0, stop=5, step=1)
True
jiggle
RangeIndex(start=0, stop=5, step=1)
True
dozen
RangeIndex(start=0, stop=5, step=1)
True
hundred
RangeIndex(start=0, stop=5, step=1)
True
imagine
RangeIndex(start=0, stop=5, step=1)
True
fun
RangeIndex(start=0, stop=5, step=1)
True
each
RangeIndex(start=0, stop=5, step=1)
True
needless
RangeIndex(start=0, stop=5, step=1)
True
wasted
RangeIndex(start=0, stop=5, step=1)
True
waste
RangeIndex(start=0, stop=5, step=1)
True
seperated
RangeIndex(start=0, stop=5, step=1)
True
mere
RangeIndex(start=0, stop=5, step=1)
True
ft
RangeIndex(start=0, stop=5, step=1)
True
excess